# 精度の調整
精度は英訳するとAccuracyとなるが、正解率のみに着目すると本質的な精度を見失う可能性がある。

例えば正解率が90%だと出力されたとする。理想的には罹患と非罹患との9割当たっていればいいのだが、問題は内訳で罹患が100%当たっていてサンプル数が90あり非罹患が一つも当たっていなくサンプル数が10でも正解率は90%となる。

そこで本章では再現率と適合率を用いて精度の調節を行う。先ほどの例のような極端なデータにはならないが混合行列や再現率・適合率の値がしきい値によって調節できるためそこの部分を確認する。

## ライブラリのインポート

In [1]:
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import numpy as np
import statsmodels.api as sm

## データの読み込み
今回は適合率と再現率を検証したいため2値分類のデータで考える

In [2]:
df = pd.read_csv("breast_cancer.csv")
df.head()

,y,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,0.0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,0.0,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,0.0,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,0.0,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,0.0,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


## 目的変数と説明変数を分ける

In [3]:
y = df["y"]
x = sm.add_constant(df.drop("y", axis=1))

## ロジスティック回帰モデルを使用

In [4]:
model = sm.Logit(y, x).fit_regularized()

C:\Users\decar\anaconda\lib\site-packages\statsmodels\discrete\discrete_model.py:1819: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
C:\Users\decar\anaconda\lib\site-packages\statsmodels\discrete\discrete_model.py:1872: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.03148814694005786
            Iterations: 503
            Function evaluations: 518
            Gradient evaluations: 503


In [5]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                  569
Model:                          Logit   Df Residuals:                      538
Method:                           MLE   Df Model:                           30
Date:                Thu, 12 Oct 2023   Pseudo R-squ.:                  0.9523
Time:                        14:19:59   Log-Likelihood:                -17.917
converged:                       True   LL-Null:                       -375.72
Covariance Type:            nonrobust   LLR p-value:                2.728e-131
===========================================================================================
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                       8.9629     52.559      0.171      0.865     -94.050     111.976
mean radius                 6.9979     21.339      0.328      0.743     -34.826      48.821
mean texture                0.0224      0.626      0.036      0.971      -1.204       1.249
mean perimeter              1.3387      3.039      0.441      0.660      -4.617       7.294
mean area                  -0.1109      0.124     -0.892      0.372      -0.354       0.133
mean smoothness          -203.6587    262.736     -0.775      0.438    -718.612     311.295
mean compactness          141.4334    142.778      0.991      0.322    -138.406     421.273
mean concavity           -120.8278    117.737     -1.026      0.305    -351.589     109.933
mean concave points      -142.7686    172.404     -0.828      0.408    -480.674     195.137
mean symmetry              84.6226     70.033      1.208      0.227     -52.639     221.884
mean fractal dimension   -202.9288    408.504     -0.497      0.619   -1003.582     597.725
radius error              -50.5151     71.556     -0.706      0.480    -190.763      89.733
texture error               5.0310      4.029      1.249      0.212      -2.866      12.928
perimeter error             7.4456      6.748      1.103      0.270      -5.780      20.671
area error                 -0.3009      0.519     -0.580      0.562      -1.317       0.716
smoothness error         -539.1715   1117.271     -0.483      0.629   -2728.982    1650.639
compactness error         -61.6349    261.563     -0.236      0.814    -574.289     451.019
concavity error           335.4967    175.117      1.916      0.055      -7.726     678.720
concave points error    -1333.5654    827.872     -1.611      0.107   -2956.165     289.035
symmetry error            319.7980    401.955      0.796      0.426    -468.019    1107.615
fractal dimension error   606.8455   1503.014      0.404      0.686   -2339.008    3552.699
worst radius                2.0029      7.622      0.263      0.793     -12.937      16.943
worst texture              -1.0247      0.620     -1.653      0.098      -2.240       0.191
worst perimeter            -0.7826      0.909     -0.861      0.389      -2.564       0.999
worst area                 -0.0366      0.068     -0.540      0.589      -0.169       0.096
worst smoothness           94.9065    170.653      0.556      0.578    -239.568     429.381
worst compactness          17.6550     48.021      0.368      0.713     -76.465     111.775
worst concavity           -30.0498     34.208     -0.878      0.380     -97.097      36.997
worst concave points      -48.3774    100.620     -0.481      0.631    -245.589     148.834
worst symmetry            -86.5107     60.216     -1.437      0.151    -204.532      31.511
worst fractal dimension  -120.0636    182.543     -0.658      0.511    -477.841     237.714
===========================================================================================

Possibly complete qu

## しきい値の設定
通常分類アルゴリズムは0.5をしきい値に設定して0.5以上を陽性、0.5未満を陰性とするが、ここでは精度の調整のため0.1刻みで予測結果を確認する。

In [6]:
pred = model.predict(x)
y_pred1 = np.where(pred >= 0.1, 1, 0)
y_pred2 = np.where(pred >= 0.2, 1, 0)
y_pred3 = np.where(pred >= 0.3, 1, 0)
y_pred4 = np.where(pred >= 0.4, 1, 0)
y_pred5 = np.where(pred >= 0.5, 1, 0)
y_pred6 = np.where(pred >= 0.6, 1, 0)
y_pred7 = np.where(pred >= 0.7, 1, 0)
y_pred8 = np.where(pred >= 0.8, 1, 0)
y_pred9 = np.where(pred >= 0.9, 1, 0)

### 予測結果の確認
ここでは適合率をprecision、再現率をrecallとして見てほしい。また、再現率と適合率の調和平均をF値と呼ぶが今回はF値とAccuracyを比較して考えてみよう。

In [7]:
print(classification_report(y.values, y_pred1))
df_cm = pd.DataFrame(confusion_matrix(y, y_pred1))
df_cm.index = ["F", "T"]
df_cm.columns = ["N", "P"]
df_cm

              precision    recall  f1-score   support

         0.0       1.00      0.95      0.97       212
         1.0       0.97      1.00      0.98       357

    accuracy                           0.98       569
   macro avg       0.99      0.97      0.98       569
weighted avg       0.98      0.98      0.98       569



,N,P
F,201,11
T,0,357


In [8]:
print(classification_report(y.values, y_pred2))
df_cm = pd.DataFrame(confusion_matrix(y, y_pred2))
df_cm.index = ["F", "T"]
df_cm.columns = ["N", "P"]
df_cm

              precision    recall  f1-score   support

         0.0       1.00      0.97      0.98       212
         1.0       0.98      1.00      0.99       357

    accuracy                           0.99       569
   macro avg       0.99      0.98      0.99       569
weighted avg       0.99      0.99      0.99       569



,N,P
F,205,7
T,0,357


In [9]:
print(classification_report(y.values, y_pred3))
df_cm = pd.DataFrame(confusion_matrix(y, y_pred3))
df_cm.index = ["F", "T"]
df_cm.columns = ["N", "P"]
df_cm

              precision    recall  f1-score   support

         0.0       1.00      0.98      0.99       212
         1.0       0.99      1.00      0.99       357

    accuracy                           0.99       569
   macro avg       0.99      0.99      0.99       569
weighted avg       0.99      0.99      0.99       569



,N,P
F,207,5
T,0,357


In [10]:
print(classification_report(y.values, y_pred4))
df_cm = pd.DataFrame(confusion_matrix(y, y_pred4))
df_cm.index = ["F", "T"]
df_cm.columns = ["N", "P"]
df_cm

              precision    recall  f1-score   support

         0.0       1.00      0.98      0.99       212
         1.0       0.99      1.00      0.99       357

    accuracy                           0.99       569
   macro avg       0.99      0.99      0.99       569
weighted avg       0.99      0.99      0.99       569



,N,P
F,207,5
T,0,357


In [11]:
print(classification_report(y.values, y_pred5))
df_cm = pd.DataFrame(confusion_matrix(y, y_pred5))
df_cm.index = ["F", "T"]
df_cm.columns = ["N", "P"]
df_cm

              precision    recall  f1-score   support

         0.0       0.99      0.98      0.98       212
         1.0       0.99      0.99      0.99       357

    accuracy                           0.99       569
   macro avg       0.99      0.99      0.99       569
weighted avg       0.99      0.99      0.99       569



,N,P
F,207,5
T,2,355


In [12]:
print(classification_report(y.values, y_pred6))
df_cm = pd.DataFrame(confusion_matrix(y, y_pred6))
df_cm.index = ["F", "T"]
df_cm.columns = ["N", "P"]
df_cm

              precision    recall  f1-score   support

         0.0       0.98      0.98      0.98       212
         1.0       0.99      0.99      0.99       357

    accuracy                           0.99       569
   macro avg       0.98      0.98      0.98       569
weighted avg       0.99      0.99      0.99       569



,N,P
F,208,4
T,4,353


In [13]:
print(classification_report(y.values, y_pred7))
df_cm = pd.DataFrame(confusion_matrix(y, y_pred7))
df_cm.index = ["F", "T"]
df_cm.columns = ["N", "P"]
df_cm

              precision    recall  f1-score   support

         0.0       0.97      0.98      0.98       212
         1.0       0.99      0.98      0.99       357

    accuracy                           0.98       569
   macro avg       0.98      0.98      0.98       569
weighted avg       0.98      0.98      0.98       569



,N,P
F,208,4
T,6,351


In [14]:
print(classification_report(y.values, y_pred8))
df_cm = pd.DataFrame(confusion_matrix(y, y_pred8))
df_cm.index = ["F", "T"]
df_cm.columns = ["N", "P"]
df_cm

              precision    recall  f1-score   support

         0.0       0.95      0.99      0.97       212
         1.0       0.99      0.97      0.98       357

    accuracy                           0.98       569
   macro avg       0.97      0.98      0.98       569
weighted avg       0.98      0.98      0.98       569



,N,P
F,210,2
T,10,347


In [15]:
print(classification_report(y.values, y_pred9))
df_cm = pd.DataFrame(confusion_matrix(y, y_pred9))
df_cm.index = ["F", "T"]
df_cm.columns = ["N", "P"]
df_cm

              precision    recall  f1-score   support

         0.0       0.94      1.00      0.97       212
         1.0       1.00      0.96      0.98       357

    accuracy                           0.97       569
   macro avg       0.97      0.98      0.97       569
weighted avg       0.97      0.97      0.97       569



,N,P
F,211,1
T,14,343


これらの結果から再現率と適合率から考える適切なしきい値は0.3または0.4であることが分かる。

このように分類問題である場合、しきい値を設定することで本質的な精度をより高めることができる。